# 00: DGX Spark CUDA Setup

**This notebook is specifically designed for NVIDIA DGX Spark systems.**

This notebook should be run **BEFORE** `01_Setup_Environment.ipynb` if you're on a DGX Spark system. It ensures PyTorch is installed with proper CUDA support for the DGX Spark's GPU.

## What This Notebook Does

1. **Detects DGX Spark System**: Identifies ARM64 architecture and NVIDIA GB10 GPU
2. **Checks Current PyTorch**: Determines if CPU-only or CUDA-enabled version is installed
3. **Installs CUDA-Enabled PyTorch**: Installs PyTorch with CUDA 13.0 support (cu130)
4. **Verifies GPU Access**: Confirms PyTorch can see and use the GPU
5. **Provides System Information**: Shows GPU details, CUDA version, and PyTorch configuration

## About DGX Spark

The **NVIDIA DGX Spark** is a compact AI workstation featuring:
- **Grace CPU**: ARM64-based processor (aarch64 architecture)
- **NVIDIA GB10 GPU**: Blackwell architecture with 128 GB unified memory
- **Compute Capability**: SM_121 (12.1) - newer than older PyTorch builds recognized
- **CUDA Version**: 13.0

**Why This Notebook is Needed:**
- Default `pip install torch` installs CPU-only version on ARM64
- DGX Spark requires CUDA 13.0 enabled PyTorch (cu130 index)
- PyTorch 2.9+ has official support for CUDA 13.0 and DGX Spark
- Without proper setup, GPU acceleration won't work

**After This Notebook:**
- Proceed to `01_Setup_Environment.ipynb` for full environment setup
- Your PyTorch will have GPU support enabled


In [2]:
# Setup: Import system utilities and check architecture
# We need to detect if we're on a DGX Spark system and check current PyTorch installation
import sys
import os
import subprocess
import platform
from pathlib import Path

# Get project root (for path management)
# Notebooks are in the repository root directory
# This handles both cases: if notebook is in root or in a subdirectory
current_dir = Path(os.getcwd())
if current_dir.name == "notebooks":
    project_root = current_dir.parent
else:
    project_root = current_dir

print("=" * 70)
print("DGX SPARK CUDA SETUP")
print("=" * 70)
print(f"\nProject root: {project_root}")
print(f"Python version: {sys.version.split()[0]}")
print(f"Python executable: {sys.executable}")
print(f"Platform: {platform.platform()}")
print(f"Architecture: {platform.machine()}")


DGX SPARK CUDA SETUP

Project root: /home/goble54/spark-dev-workspace/Lora-for-embedding-models
Python version: 3.12.3
Python executable: /home/goble54/spark-dev-workspace/Lora-for-embedding-models/.venv/bin/python
Platform: Linux-6.11.0-1016-nvidia-aarch64-with-glibc2.39
Architecture: aarch64


## Step 1: Detect DGX Spark System

We'll check if this is a DGX Spark system by examining:
- Architecture (should be ARM64/aarch64)
- GPU presence (NVIDIA GB10)
- CUDA version (should be 13.0)


In [3]:
# Check system architecture
# DGX Spark uses ARM64 (aarch64) architecture, not x86_64
# This is important because PyTorch builds differ by architecture
architecture = platform.machine()
is_arm64 = architecture == "aarch64" or architecture == "arm64"

print(f"System Architecture: {architecture}")
if is_arm64:
    print("✓ ARM64 architecture detected (DGX Spark uses ARM64)")
else:
    print(f"⚠ Expected ARM64 (aarch64) for DGX Spark, got {architecture}")
    print("  This notebook is designed for DGX Spark, but may work on other systems")

# Check for NVIDIA GPU using nvidia-smi
# This confirms we have an NVIDIA GPU (DGX Spark has GB10)
gpu_detected = False
gpu_name = None
cuda_version = None

try:
    # Run nvidia-smi to get GPU information
    # nvidia-smi is the standard NVIDIA tool for GPU information
    # Why nvidia-smi? Works on all NVIDIA systems, shows GPU name and CUDA version
    result = subprocess.run(
        ['nvidia-smi', '--query-gpu=name,driver_version,compute_cap', '--format=csv,noheader'],
        capture_output=True,
        text=True,
        timeout=10
    )
    
    if result.returncode == 0 and result.stdout.strip():
        # Parse GPU information
        # Format: "GPU Name, Driver Version, Compute Capability"
        gpu_info = result.stdout.strip().split('\n')[0]  # Get first GPU
        parts = [p.strip() for p in gpu_info.split(',')]
        if len(parts) >= 3:
            gpu_name = parts[0]
            driver_version = parts[1]
            compute_cap = parts[2]
            gpu_detected = True
            
            print(f"\n✓ NVIDIA GPU Detected:")
            print(f"  GPU Name: {gpu_name}")
            print(f"  Driver Version: {driver_version}")
            print(f"  Compute Capability: {compute_cap}")
            
            # Check if it's GB10 (DGX Spark's GPU)
            if "GB10" in gpu_name.upper():
                print("  ✓ This appears to be a DGX Spark (GB10 GPU)")
            else:
                print(f"  ⚠ GPU is not GB10 (expected for DGX Spark)")
except (FileNotFoundError, subprocess.TimeoutExpired) as e:
    print(f"\n⚠ nvidia-smi not available or timed out")
    print("  This might not be a DGX Spark system, or NVIDIA drivers not installed")

# Get CUDA version from nvidia-smi
# This shows the CUDA version supported by the driver
try:
    result = subprocess.run(['nvidia-smi'], capture_output=True, text=True, timeout=5)
    if result.returncode == 0:
        for line in result.stdout.split('\n'):
            if 'CUDA Version:' in line:
                # Extract CUDA version (e.g., "13.0" from "CUDA Version: 13.0")
                parts = line.split('CUDA Version:')
                if len(parts) > 1:
                    cuda_version = parts[1].strip().split()[0]
                    print(f"\n✓ System CUDA Version: {cuda_version}")
                    break
except Exception:
    pass

if not gpu_detected:
    print("\n⚠ No NVIDIA GPU detected!")
    print("  This notebook is designed for DGX Spark systems with NVIDIA GPUs")
    print("  You can still proceed, but GPU acceleration won't be available")


System Architecture: aarch64
✓ ARM64 architecture detected (DGX Spark uses ARM64)

✓ NVIDIA GPU Detected:
  GPU Name: NVIDIA GB10
  Driver Version: 580.95.05
  Compute Capability: 12.1
  ✓ This appears to be a DGX Spark (GB10 GPU)

✓ System CUDA Version: 13.0


## Step 2: Check Current PyTorch Installation

We need to check if PyTorch is already installed and whether it has CUDA support. On DGX Spark, the default `pip install torch` often installs a CPU-only version, which won't use the GPU.


In [4]:
# Try to import PyTorch and check its configuration
# This tells us if PyTorch is installed and whether it has CUDA support
pytorch_installed = False
pytorch_has_cuda = False
pytorch_version = None
pytorch_cuda_version = None

try:
    import torch
    pytorch_installed = True
    pytorch_version = torch.__version__
    pytorch_has_cuda = torch.version.cuda is not None
    pytorch_cuda_version = torch.version.cuda if pytorch_has_cuda else None
    
    print("Current PyTorch Installation:")
    print(f"  Installed: Yes")
    print(f"  Version: {pytorch_version}")
    print(f"  CUDA Support: {'Yes' if pytorch_has_cuda else 'No (CPU-only)'}")
    if pytorch_has_cuda:
        print(f"  CUDA Version: {pytorch_cuda_version}")
    
    # Check if GPU is accessible
    if pytorch_has_cuda:
        cuda_available = torch.cuda.is_available()
        device_count = torch.cuda.device_count() if cuda_available else 0
        
        print(f"\nGPU Access:")
        print(f"  CUDA Available: {cuda_available}")
        if cuda_available:
            print(f"  GPU Count: {device_count}")
            if device_count > 0:
                gpu_name_pytorch = torch.cuda.get_device_name(0)
                print(f"  GPU Name: {gpu_name_pytorch}")
                print("  ✓ PyTorch can access the GPU!")
            else:
                print("  ⚠ No GPUs detected by PyTorch")
        else:
            print("  ⚠ CUDA not available (driver or runtime issue)")
    else:
        print("\n⚠ PyTorch is CPU-only - GPU acceleration not available")
        print("  This is the common issue on DGX Spark!")
        print("  We'll fix this in the next step.")
        
except ImportError:
    print("Current PyTorch Installation:")
    print("  Installed: No")
    print("  PyTorch is not installed yet")
    print("  We'll install it with CUDA support in the next step")
except Exception as e:
    print(f"Error checking PyTorch: {e}")

# Determine if we need to reinstall
needs_installation = not pytorch_installed
needs_reinstall = pytorch_installed and not pytorch_has_cuda

if needs_reinstall:
    print("\n" + "=" * 70)
    print("ACTION NEEDED: PyTorch CPU-only version detected")
    print("=" * 70)
    print("We need to:")
    print("  1. Uninstall CPU-only PyTorch")
    print("  2. Install PyTorch with CUDA 13.0 support (cu130)")
    print("=" * 70)
elif needs_installation:
    print("\n" + "=" * 70)
    print("ACTION NEEDED: PyTorch not installed")
    print("=" * 70)
    print("We'll install PyTorch with CUDA 13.0 support")
    print("=" * 70)
else:
    print("\n" + "=" * 70)
    print("✓ PyTorch with CUDA is already installed!")
    print("=" * 70)
    print("You can skip the installation step and proceed to notebook 01")
    print("=" * 70)


Current PyTorch Installation:
  Installed: Yes
  Version: 2.9.1+cpu
  CUDA Support: No (CPU-only)

⚠ PyTorch is CPU-only - GPU acceleration not available
  This is the common issue on DGX Spark!
  We'll fix this in the next step.

ACTION NEEDED: PyTorch CPU-only version detected
We need to:
  1. Uninstall CPU-only PyTorch
  2. Install PyTorch with CUDA 13.0 support (cu130)


## Step 3: Install PyTorch with CUDA 13.0 Support

For DGX Spark, we need to install PyTorch from the CUDA 13.0 wheel index (`cu130`). This provides:
- CUDA 13.0 support (matches your system CUDA version)
- ARM64 (aarch64) builds for DGX Spark's Grace CPU
- Support for GB10 GPU (compute capability 12.1)

**Why cu130 index?**
- PyTorch 2.9+ has official CUDA 13.0 support
- Default pip install gives CPU-only on ARM64
- cu130 index provides GPU-enabled builds for ARM64
- Matches DGX Spark's CUDA 13.0 system

**Installation Process:**
1. Uninstall existing CPU-only PyTorch (if present)
2. Install from cu130 index: `pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu130`
3. Verify installation

**Important:** After installation, you MUST restart the kernel for changes to take effect!


In [5]:
# Installation configuration
# Set this to True to actually install PyTorch with CUDA
# Set to False to just see what would be installed
INSTALL_PYTORCH = True  # Change to True to install

# Determine which pip to use
# We want to use the virtual environment's pip if it exists
# Otherwise use system pip
venv_path = project_root / ".venv"
if venv_path.exists():
    if platform.system() == "Windows":
        pip_cmd = [str(venv_path / "Scripts" / "pip")]
    else:
        pip_cmd = [str(venv_path / "bin" / "pip")]
    print(f"Using virtual environment pip: {pip_cmd[0]}")
else:
    pip_cmd = [sys.executable, "-m", "pip"]
    print(f"Using system pip: {' '.join(pip_cmd)}")

# Build installation command for CUDA 13.0
# This is the correct command for DGX Spark
# Why cu130? PyTorch 2.9+ supports CUDA 13.0, and DGX Spark uses CUDA 13.0
cuda_index_url = "https://download.pytorch.org/whl/cu130"
install_cmd = pip_cmd + [
    "install",
    "--upgrade",  # Upgrade if already installed
    "torch",
    "torchvision", 
    "torchaudio",
    "--index-url", cuda_index_url
]

print("\n" + "=" * 70)
print("PYTORCH INSTALLATION COMMAND")
print("=" * 70)
print(" ".join(install_cmd))
print("=" * 70)

if INSTALL_PYTORCH:
    print("\n" + "=" * 70)
    print("INSTALLING PyTorch with CUDA 13.0 support...")
    print("=" * 70)
    print("⚠ This will take several minutes (downloading ~2GB of packages)")
    print("⚠ After installation, RESTART THE KERNEL (Kernel → Restart Kernel)")
    print("=" * 70)
    
    # Step 1: Uninstall existing PyTorch if CPU-only
    # Why uninstall first? Prevents conflicts, ensures clean installation
    if needs_reinstall:
        print("\nStep 1: Uninstalling CPU-only PyTorch...")
        uninstall_cmd = pip_cmd + ["uninstall", "-y", "torch", "torchvision", "torchaudio"]
        result = subprocess.run(uninstall_cmd, capture_output=True, text=True)
        if result.returncode == 0:
            print("✓ CPU-only PyTorch uninstalled")
        else:
            print(f"⚠ Uninstall had issues: {result.stderr}")
    
    # Step 2: Install PyTorch with CUDA 13.0
    print("\nStep 2: Installing PyTorch with CUDA 13.0 support...")
    print("This downloads and installs:")
    print("  - torch (PyTorch core with CUDA 13.0)")
    print("  - torchvision (computer vision utilities)")
    print("  - torchaudio (audio processing utilities)")
    print("  - NVIDIA CUDA libraries (cudnn, cublas, etc.)")
    print("\nDownloading... (this may take 5-10 minutes)")
    
    result = subprocess.run(install_cmd, capture_output=False, text=True)
    
    if result.returncode == 0:
        print("\n" + "=" * 70)
        print("✓ PyTorch with CUDA 13.0 installed successfully!")
        print("=" * 70)
        print("\n⚠ CRITICAL NEXT STEPS:")
        print("  1. Go to: Kernel → Restart Kernel (or Kernel → Restart & Clear Output)")
        print("  2. Re-run this notebook from the beginning")
        print("  3. Verify GPU detection in Step 4")
        print("  4. Then proceed to notebook 01_Setup_Environment.ipynb")
        print("=" * 70)
    else:
        print("\n" + "=" * 70)
        print("✗ Installation failed")
        print("=" * 70)
        print("\nTroubleshooting:")
        print("  1. Check internet connection (needs to download ~2GB)")
        print("  2. Ensure you have enough disk space")
        print("  3. Try running in terminal:")
        print(f"     {' '.join(install_cmd)}")
        print("=" * 70)
else:
    print("\n" + "=" * 70)
    print("INSTALLATION NOT RUN (INSTALL_PYTORCH = False)")
    print("=" * 70)
    print("\nTo install PyTorch with CUDA 13.0:")
    print("  1. Change INSTALL_PYTORCH = True in the cell above")
    print("  2. Re-run this cell")
    print("  3. After installation, restart kernel and re-run all cells")
    print("=" * 70)


Using virtual environment pip: /home/goble54/spark-dev-workspace/Lora-for-embedding-models/.venv/bin/pip

PYTORCH INSTALLATION COMMAND
/home/goble54/spark-dev-workspace/Lora-for-embedding-models/.venv/bin/pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu130

INSTALLING PyTorch with CUDA 13.0 support...
⚠ This will take several minutes (downloading ~2GB of packages)
⚠ After installation, RESTART THE KERNEL (Kernel → Restart Kernel)

Step 1: Uninstalling CPU-only PyTorch...
✓ CPU-only PyTorch uninstalled

Step 2: Installing PyTorch with CUDA 13.0 support...
This downloads and installs:
  - torch (PyTorch core with CUDA 13.0)
  - torchvision (computer vision utilities)
  - torchaudio (audio processing utilities)
  - NVIDIA CUDA libraries (cudnn, cublas, etc.)

Downloading... (this may take 5-10 minutes)
Looking in indexes: https://download.pytorch.org/whl/cu130
  Using cached https://download.pytorch.org/whl/cu130/torch-2.9.1%2Bcu130-cp312-

## Step 4: Verify GPU Detection

After installing PyTorch with CUDA support and restarting the kernel, run this cell to verify that PyTorch can see and use your GPU.

**What to expect:**
- PyTorch version should show `+cu130` (not `+cpu`)
- `torch.cuda.is_available()` should return `True`
- GPU name should be `NVIDIA GB10` (or similar)
- No warnings about compute capability


In [1]:
# Verify PyTorch installation and GPU access
# This should be run AFTER installing PyTorch and restarting the kernel
# If you see errors, make sure you restarted the kernel after installation

try:
    import torch
    
    print("=" * 70)
    print("PYTORCH GPU VERIFICATION")
    print("=" * 70)
    
    # Check PyTorch version
    pytorch_version = torch.__version__
    print(f"\nPyTorch Version: {pytorch_version}")
    
    # Check if CUDA is compiled in
    pytorch_has_cuda = torch.version.cuda is not None
    pytorch_cuda_version = torch.version.cuda if pytorch_has_cuda else None
    
    if pytorch_has_cuda:
        print(f"✓ CUDA Support: Yes (version {pytorch_cuda_version})")
        
        # Check if CUDA runtime is available
        cuda_available = torch.cuda.is_available()
        print(f"CUDA Available: {cuda_available}")
        
        if cuda_available:
            # Get GPU information
            device_count = torch.cuda.device_count()
            print(f"GPU Count: {device_count}")
            
            if device_count > 0:
                for i in range(device_count):
                    gpu_name = torch.cuda.get_device_name(i)
                    gpu_props = torch.cuda.get_device_properties(i)
                    
                    print(f"\nGPU {i}: {gpu_name}")
                    print(f"  Total Memory: {gpu_props.total_memory / (1024**3):.1f} GB")
                    print(f"  Compute Capability: {gpu_props.major}.{gpu_props.minor}")
                    
                    # Check if it's GB10 (DGX Spark)
                    if "GB10" in gpu_name.upper():
                        print("  ✓ This is the DGX Spark GB10 GPU!")
                
                # Test GPU computation
                print("\nTesting GPU computation...")
                try:
                    # Create a small tensor on GPU to verify it works
                    # This is a simple test: create random tensor, move to GPU, compute sum
                    test_tensor = torch.randn(1000, 1000)
                    test_tensor_gpu = test_tensor.cuda()  # Move to GPU
                    result = test_tensor_gpu.sum()  # Compute on GPU
                    print(f"✓ GPU computation test passed!")
                    print(f"  Test result: {result.item():.4f}")
                    print(f"  Device used: {test_tensor_gpu.device}")
                except Exception as e:
                    print(f"✗ GPU computation test failed: {e}")
                    print("  This suggests a problem with GPU access")
                
                print("\n" + "=" * 70)
                print("✓ SUCCESS: GPU is working with PyTorch!")
                print("=" * 70)
                print("\nYou can now proceed to:")
                print("  - Notebook 01_Setup_Environment.ipynb (full environment setup)")
                print("  - Your GPU will be used for all PyTorch operations")
                print("=" * 70)
            else:
                print("\n⚠ No GPUs detected by PyTorch")
                print("  Check CUDA drivers: nvidia-smi")
        else:
            print("\n⚠ CUDA not available")
            print("  Possible causes:")
            print("    - CUDA drivers not installed")
            print("    - CUDA version mismatch")
            print("    - GPU in use by another process")
            print("  Check: nvidia-smi")
    else:
        print(f"✗ CUDA Support: No (CPU-only version)")
        print("\n⚠ PyTorch is still CPU-only!")
        print("  Did you:")
        print("    1. Run the installation cell (Step 3)?")
        print("    2. Restart the kernel after installation?")
        print("    3. Re-run all cells from the beginning?")
        print("\n  If not, go back to Step 3 and install PyTorch with CUDA")
        
except ImportError:
    print("=" * 70)
    print("✗ PyTorch not installed")
    print("=" * 70)
    print("\nGo to Step 3 to install PyTorch with CUDA support")
except Exception as e:
    print(f"Error verifying PyTorch: {e}")
    import traceback
    traceback.print_exc()


PYTORCH GPU VERIFICATION

PyTorch Version: 2.9.1+cu130
✓ CUDA Support: Yes (version 13.0)
CUDA Available: True
GPU Count: 1

GPU 0: NVIDIA GB10
  Total Memory: 119.7 GB
  Compute Capability: 12.1
  ✓ This is the DGX Spark GB10 GPU!

Testing GPU computation...
✓ GPU computation test passed!
  Test result: 567.8577
  Device used: cuda:0

✓ SUCCESS: GPU is working with PyTorch!

You can now proceed to:
  - Notebook 01_Setup_Environment.ipynb (full environment setup)
  - Your GPU will be used for all PyTorch operations


/home/goble54/spark-dev-workspace/Lora-for-embedding-models/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:283: UserWarning: 
    Found GPU0 NVIDIA GB10 which is of cuda capability 12.1.
    Minimum and Maximum cuda capability supported by this version of PyTorch is
    (8.0) - (12.0)
    
  warnings.warn(


## Step 5: System Information Summary

This cell provides a comprehensive summary of your DGX Spark system configuration for reference.


In [2]:
# Generate comprehensive system report
# This documents your DGX Spark configuration for reference
# Note: Re-import modules in case kernel was restarted
import sys
import platform
from pathlib import Path

print("=" * 70)
print("DGX SPARK SYSTEM REPORT")
print("=" * 70)

# System information
print("\n🖥️  System Information:")
print(f"  Architecture: {platform.machine()}")
print(f"  Platform: {platform.platform()}")
print(f"  Python: {sys.version.split()[0]} ({sys.executable})")

# GPU information (with defensive checks)
# Check if variables from earlier cells exist
gpu_detected = globals().get('gpu_detected', False)
gpu_name = globals().get('gpu_name', None)
cuda_version = globals().get('cuda_version', None)

if gpu_detected and gpu_name:
    print(f"\n🎮 GPU Information:")
    print(f"  GPU Name: {gpu_name}")
    if cuda_version:
        print(f"  System CUDA: {cuda_version}")
else:
    print(f"\n🎮 GPU Information:")
    print(f"  Status: Not detected (run Step 1 cells to detect GPU)")
    print(f"  Note: Run cells from Step 1 if you haven't already")

# PyTorch information
try:
    import torch
    print(f"\n🔥 PyTorch Information:")
    print(f"  Version: {torch.__version__}")
    print(f"  CUDA Compiled: {'Yes' if torch.version.cuda else 'No'}")
    if torch.version.cuda:
        print(f"  CUDA Version: {torch.version.cuda}")
        if torch.cuda.is_available():
            print(f"  CUDA Available: Yes")
            print(f"  GPU Count: {torch.cuda.device_count()}")
            if torch.cuda.device_count() > 0:
                print(f"  GPU Name: {torch.cuda.get_device_name(0)}")
        else:
            print(f"  CUDA Available: No")
    else:
        print(f"  Status: CPU-only (needs CUDA installation)")
except ImportError:
    print(f"\n🔥 PyTorch Information:")
    print(f"  Status: Not installed")

# Installation status (with defensive checks)
needs_installation = globals().get('needs_installation', None)
needs_reinstall = globals().get('needs_reinstall', None)

print(f"\n📦 Installation Status:")
if needs_installation is True:
    print(f"  PyTorch: Not installed")
    print(f"  Action: Install with CUDA 13.0 support")
elif needs_reinstall is True:
    print(f"  PyTorch: Installed (CPU-only)")
    print(f"  Action: Reinstall with CUDA 13.0 support")
elif needs_installation is False and needs_reinstall is False:
    try:
        import torch
        if torch.cuda.is_available():
            print(f"  PyTorch: Installed with CUDA ✓")
            print(f"  Action: Ready to use!")
        else:
            print(f"  PyTorch: Installed with CUDA (but GPU not accessible)")
            print(f"  Action: Check CUDA drivers")
    except:
        print(f"  PyTorch: Unknown status")
else:
    # Variables not set - cells from Step 2 weren't run
    print(f"  Status: Unknown (run Step 2 cells to check PyTorch installation)")
    print(f"  Note: Run cells from Step 2 if you haven't already")

print("\n" + "=" * 70)
print("Report complete!")
print("=" * 70)
print("\nNote: If some information is missing, make sure to run all cells")
print("      from Steps 1-4 in order for complete system information.")


DGX SPARK SYSTEM REPORT

🖥️  System Information:
  Architecture: aarch64
  Platform: Linux-6.11.0-1016-nvidia-aarch64-with-glibc2.39
  Python: 3.12.3 (/home/goble54/spark-dev-workspace/Lora-for-embedding-models/.venv/bin/python)

🎮 GPU Information:
  Status: Not detected (run Step 1 cells to detect GPU)
  Note: Run cells from Step 1 if you haven't already

🔥 PyTorch Information:
  Version: 2.9.1+cu130
  CUDA Compiled: Yes
  CUDA Version: 13.0
  CUDA Available: Yes
  GPU Count: 1
  GPU Name: NVIDIA GB10

📦 Installation Status:
  Status: Unknown (run Step 2 cells to check PyTorch installation)
  Note: Run cells from Step 2 if you haven't already

Report complete!

Note: If some information is missing, make sure to run all cells
      from Steps 1-4 in order for complete system information.


## Summary and Next Steps

**What We Accomplished:**
1. ✅ Detected DGX Spark system (ARM64 + GB10 GPU)
2. ✅ Checked current PyTorch installation
3. ✅ Installed PyTorch with CUDA 13.0 support (if needed)
4. ✅ Verified GPU detection and access
5. ✅ Generated system configuration report

**Key Points for DGX Spark:**
- **Architecture**: ARM64 (aarch64) - different from x86_64
- **GPU**: NVIDIA GB10 (Blackwell, compute capability 12.1)
- **CUDA**: 13.0 - use `cu130` PyTorch index
- **PyTorch**: 2.9+ has official DGX Spark support

**Common Issues and Solutions:**

1. **CPU-only PyTorch installed:**
   - Solution: Uninstall and reinstall from `cu130` index
   - Command: `pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu130`

2. **GPU not detected after installation:**
   - Solution: Restart kernel (Kernel → Restart Kernel)
   - Then re-run verification cell

3. **Compute capability warnings:**
   - These are informational (GB10 is 12.1, PyTorch supports up to 12.0)
   - GPU will still work, may use compatibility mode
   - Future PyTorch versions will fully support 12.1

**Next Steps:**
1. If GPU is working: Proceed to `01_Setup_Environment.ipynb`
2. If issues persist: Check NVIDIA drivers (`nvidia-smi`)
3. For optimal performance: Consider using NVIDIA's prebuilt containers

**References:**
- PyTorch CUDA 13.0 wheels: https://download.pytorch.org/whl/cu130/
- DGX Spark documentation: NVIDIA Developer Forums
- PyTorch DGX Spark support: https://discuss.pytorch.org/t/nvidia-dgx-spark-support/
